# Práctica semanal

## Primer ejercicio: *k*-NN

Retomemos nuestro dataset geográfico de provincias

In [ ]:
# Cargamos bibliotecas importantes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing

# Cargamos el dataset de provincias
provincias_data = pd.read_csv('../data/provincias_scaled.csv')

# Y mostramos los puntos en el mapa
# Primero, codifiquemos las provincias en 0-6
le = preprocessing.LabelEncoder()
le.fit(provincias_data['provincia'])
c = le.transform(provincias_data['provincia'])

# Leemos el archivo
im = plt.imread('../data/mapaCR.png')
implot = plt.imshow(im)

# Escalamos los arrays en 0-1
p_long = provincias_data['long']
p_lat = provincias_data['lat']

# Dibujamos (con un pequeño offset empírico para que caiga en la imágen)
plt.scatter((im.shape[0]*p_long)*0.95 + 25, (im.shape[1]*(1-p_lat))*0.8 + 20, c=c)
plt.show()

Vamos a probar distintos valores de $k$ en nuestro clasificador *k*-NN para ver su efecto en un dataset multiclase.

In [ ]:
# Obtenemos la envolvente convexa del territorio nacional
from scipy.spatial import Delaunay
points = np.array((p_long, p_lat)).T
hull = Delaunay(points)

# Generaremos un grid de prueba filtrado por la envolvente convexa
XX, YY = np.meshgrid(np.linspace(0, 1, 100), np.linspace(0, 1, 100))
pos = np.vstack([XX.ravel(), YY.ravel()]).T
samples = np.array([x for x in pos if hull.find_simplex(x) >= 0]).T

In [ ]:
### EJERCICIO 1 (5pt)
### Entrenar modelos de clasificación KNN para k={1, 3, 5, 10, 30, 100, 200}
from sklearn.neighbors import KNeighborsClassifier

## Entrenaremos los modelos usando los datos 'points' contra la variable provincias_data['provincia'] como target

knn_1 = KNeighborsClassifier(n_neighbors=1).fit(points, provincias_data['provincia'])
knn_3 = KNeighborsClassifier(n_neighbors=3).fit(points, provincias_data['provincia'])
knn_5 = KNeighborsClassifier(n_neighbors=5).fit(points, provincias_data['provincia'])
knn_10 = KNeighborsClassifier(n_neighbors=10).fit(points, provincias_data['provincia'])
knn_30 = KNeighborsClassifier(n_neighbors=30).fit(points, provincias_data['provincia'])
knn_100 = KNeighborsClassifier(n_neighbors=100).fit(points, provincias_data['provincia'])
knn_200 = KNeighborsClassifier(n_neighbors=200).fit(points, provincias_data['provincia'])

## Luego, evaluaremos las predicciones con los valores del array "samples"
preds_knn_1 = knn_1.predict(samples.T)
preds_knn_3 = knn_3.predict(samples.T)
preds_knn_5 = knn_5.predict(samples.T)
preds_knn_10 = knn_10.predict(samples.T)
preds_knn_30 = knn_30.predict(samples.T)
preds_knn_100 = knn_100.predict(samples.T)
preds_knn_200 = knn_200.predict(samples.T)

knn = KNeighborsClassifier(n_neighbors=2)
knn.fit(points, provincias_data['provincia'])
preds_knn = knn.predict(samples.T)

In [ ]:
### Utilice esta celda para probar los resultados de sus modelos ###

im = plt.imread('../data/mapaCR.png')
implot = plt.imshow(im)
plt.scatter((samples[0]*im.shape[0])*0.95 + 25, ((1.0-samples[1])*im.shape[1])*0.8 + 20,c=le.transform(preds_knn_1), alpha=0.1)
plt.title("k=1")
plt.show()

im = plt.imread('../data/mapaCR.png')
implot = plt.imshow(im)
plt.scatter((samples[0]*im.shape[0])*0.95 + 25, ((1.0-samples[1])*im.shape[1])*0.8 + 20,c=le.transform(preds_knn_3), alpha=0.1)
plt.title("k=3")
plt.show()

im = plt.imread('../data/mapaCR.png')
implot = plt.imshow(im)
plt.scatter((samples[0]*im.shape[0])*0.95 + 25, ((1.0-samples[1])*im.shape[1])*0.8 + 20,c=le.transform(preds_knn_5), alpha=0.1)
plt.title("k=5")
plt.show()

im = plt.imread('../data/mapaCR.png')
implot = plt.imshow(im)
plt.scatter((samples[0]*im.shape[0])*0.95 + 25, ((1.0-samples[1])*im.shape[1])*0.8 + 20,c=le.transform(preds_knn_10), alpha=0.1)
plt.title("k=10")
plt.show()

im = plt.imread('../data/mapaCR.png')
implot = plt.imshow(im)
plt.scatter((samples[0]*im.shape[0])*0.95 + 25, ((1.0-samples[1])*im.shape[1])*0.8 + 20,c=le.transform(preds_knn_30), alpha=0.1)
plt.title("k=30")
plt.show()

im = plt.imread('../data/mapaCR.png')
implot = plt.imshow(im)
plt.scatter((samples[0]*im.shape[0])*0.95 + 25, ((1.0-samples[1])*im.shape[1])*0.8 + 20,c=le.transform(preds_knn_100), alpha=0.1)
plt.title("k=100")
plt.show()

im = plt.imread('../data/mapaCR.png')
implot = plt.imshow(im)
plt.scatter((samples[0]*im.shape[0])*0.95 + 25, ((1.0-samples[1])*im.shape[1])*0.8 + 20,c=le.transform(preds_knn_200), alpha=0.1)
plt.title("k=200")
plt.show()

## Segundo ejercicio: reducción de la dimensionalidad

Utilizaremos un dataset llamado ``waveform``, que tiene 40 columnas numéricas para clasificar formas de onda en tres clases ``0``, ``1`` y ``2``. 

In [ ]:
# Leemos el dataset
waveform = pd.read_csv('../data/waveform.csv')

# Tomamos las columnas independientes
wave_x_all = np.array(waveform.loc[:, waveform.columns != 'class'])

# Y la dependiente
wave_y_all = np.array(waveform['class'])

# Los separamos en dos conjuntos entrenamiento/pruebas
wave_x_train = wave_x_all[:4000]
wave_y_train = wave_y_all[:4000]
wave_x_test = wave_x_all[4000:]
wave_y_test = wave_y_all[4000:]

In [ ]:
## EJERCICIO 1 (1pt): Entrenar un modelo k-NN con k=10 y reportar la precisión

knn = KNeighborsClassifier(n_neighbors=10).fit(wave_x_train, wave_y_train)
(knn.predict(wave_x_test) == wave_y_test).mean()

In [ ]:
## EJERCICIO 2 (2pt): Reducir la dimensionalidad utilizando análisis de discriminante lineal a 2 dimensiones y dibujarlo
## IMPORTANTE: utilizar el mismo modelo para reducir los datos de prueba
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis(n_components=2).fit(wave_x_train, wave_y_train)

wave_x_reduced_train = lda.transform(wave_x_train).T
wave_x_reduced_test = lda.transform(wave_x_test).T

plt.scatter(wave_x_reduced_train[0], wave_x_reduced_train[1], c=['red' if x==0 else 'blue' if x==1 else 'black' for x in wave_y_train])

In [ ]:
## EJERCICIO 3 (2pt): Entrenar un clasificador k-NN con los datos reducidos por discriminante lineal. 

knn = KNeighborsClassifier(n_neighbors=10).fit(wave_x_reduced_train.T, wave_y_train)

(knn.predict(wave_x_reduced_test.T) == wave_y_test).mean()